# Importamos librerias

In [3]:
import pandas as pd
import random
import hashlib
import pymysql

# Exploramos el dataset

## Creamos el dataframe

In [4]:
df = pd.read_csv('data_prueba_técnica.csv')
df.head()

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05


## Informacion general

In [5]:
# obtener la cantidad de filas y columnas
print(f"Shape del Dataframe: {df.shape}\n")
# Mostramos la información del dataframe
df.info()

Shape del Dataframe: (10000, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9997 non-null   object 
 1   name        9997 non-null   object 
 2   company_id  9996 non-null   object 
 3   amount      10000 non-null  float64
 4   status      10000 non-null  object 
 5   created_at  10000 non-null  object 
 6   paid_at     6009 non-null   object 
dtypes: float64(1), object(6)
memory usage: 547.0+ KB


In [6]:
# Vemos si hay valores null en el dataframe
df.isnull().sum()

id               3
name             3
company_id       4
amount           0
status           0
created_at       0
paid_at       3991
dtype: int64

Observamos que hay valores faltantes en las columnas 'id', 'company_name', 'company_id' y 'paid_at'. 

Esto será un problema ya que más adelante se usará el siguiente esquema:

```
id VARCHAR(24) PRIMARY KEY NOT NULL,
company_name VARCHAR(130) NULL
company_id VARCHAR(24) NOT NULL,
amount DECIMAL(16, 2) NOT NULL,
status VARCHAR(30) NOT NULL,
created_at TIMESTAMP NOT NULL,
updated_at TIMESTAMP NULL,
```

por lo tanto 'id' y 'company_id' no pueden ser nulos.

## Encontar valores faltantes

In [7]:
# Primero, obtenemos una copia del dataframe original
df_copy = df.copy()
df_copy.head()

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05


In [9]:
# Obtenemos la cantidad de valores unicos para las columnas 'id', 'name' y 'company_id'
print(f"Valores únicos para la columna 'id': {df_copy['id'].nunique()}")
print(f"Valores únicos para la columna 'name': {df_copy['name'].nunique()}")
print(f"Valores únicos para la columna 'company_id': {df_copy['company_id'].nunique()}")

Valores únicos para la columna 'id': 9997
Valores únicos para la columna 'name': 4
Valores únicos para la columna 'company_id': 3


In [10]:
# Obtenemos los valores únicos de la columna 'company_id'
print(df_copy['company_id'].unique())

# Calculamos la longitud de los valores únicos de la columna 'company_id'
for item in df_copy['company_id'].unique():
    try:
        print(f"Longitud de {item}: {len(item)}")
    except:
        print(f"Error: {item}")

['cbf1c8b09cd5b549416d49d220a40cbd317f952e'
 '8f642dc67fccf861548dfe1c761ce22f795e91f0' nan '*******']
Longitud de cbf1c8b09cd5b549416d49d220a40cbd317f952e: 40
Longitud de 8f642dc67fccf861548dfe1c761ce22f795e91f0: 40
Error: nan
Longitud de *******: 7


In [11]:
# Obtenemos los valores únicos de la columna 'name'
print(df_copy['name'].unique())

# Calculamos la longitud de los valores únicos de la columna 'name'
for item in df_copy['name'].unique():
    try:
        print(f"Longitud de {item}: {len(item)}")
    except:
        print(f"Error: {item}")

['MiPasajefy' 'Muebles chidos' nan 'MiPas0xFFFF' 'MiP0xFFFF']
Longitud de MiPasajefy: 10
Longitud de Muebles chidos: 14
Error: nan
Longitud de MiPas0xFFFF: 11
Longitud de MiP0xFFFF: 9


### Columna: 'id'

In [12]:
# Aqui no hay forma de obtener los valores faltantes ya que son unicos, así que se toma la decision de crearlos artificialmente
# Función para generar un hash aleatorio con el mismo formato
def generar_id():
    random_str = str(random.getrandbits(256))
    return hashlib.sha1(random_str.encode()).hexdigest()
# Reemplazar valores NaN con un ID aleatorio
df_copy['id'] = df_copy['id'].apply(lambda x: generar_id() if pd.isna(x) else x)

In [13]:
# Calculamos la longitud de los valores únicos de la columna 'id'
list_id_length = [len(item) for item in df_copy['id'].unique()]
max(list_id_length) 

40

### Columna: 'company_id'

In [14]:
# Usaremos la columna 'name' para encontar los valores faltantes en la columna 'company_id'
# Obtenemos la moda de la columna 'company_id' cuando 'name' es igual a 'MiPasajefy'
moda_MiPasajefy = df_copy[df_copy['name']=='MiPasajefy']['company_id'].mode()
# Si 'name' es igual a 'MiPasajefy' reemplazamos 'company_id' por moda_MiPasajefy
df_copy.loc[df_copy['name'] == 'MiPasajefy', 'company_id'] = moda_MiPasajefy
# Reemplazamos los valores faltantes en la columna 'company_id' por la moda solo si 'name' es igual 'MiPasajefy'
df_copy.loc[df_copy['name']=='MiPasajefy', 'company_id'] = df_copy.loc[df_copy['name']=='MiPasajefy', 'company_id'].fillna(moda_MiPasajefy[0])

In [15]:
# Obtenemos los valores únicos de la columna 'company_id' y confimamos que no hay valores faltantes
print(df_copy['company_id'].unique())
print(f"\nValores faltantes en la columna 'company_id': {df_copy['company_id'].isnull().sum()}")

['cbf1c8b09cd5b549416d49d220a40cbd317f952e'
 '8f642dc67fccf861548dfe1c761ce22f795e91f0']

Valores faltantes en la columna 'company_id': 0


### Columna: 'name'

In [16]:
# Filtramos los valores para los cuales 'name' es igual a 'MiPas0xFFFF' o 'nMiP0xFFFF' y asi obtener el 'company_id' correspondiente
print(f"MiPas0xFFFF company_id\n{df_copy[df_copy['name']=='MiPas0xFFFF']['company_id']}")
print(f"\nMiP0xFFFF company_id\n{df_copy[df_copy['name']=='MiP0xFFFF']['company_id']}")

MiPas0xFFFF company_id
1320    cbf1c8b09cd5b549416d49d220a40cbd317f952e
Name: company_id, dtype: object

MiP0xFFFF company_id
1479    cbf1c8b09cd5b549416d49d220a40cbd317f952e
Name: company_id, dtype: object


In [17]:
# Filtramos el valor para el cual 'company_id' es igual a 'cbf1c8b09cd5b549416d49d220a40cbd317f952e' y mostramos la moda de la columna 'name'
df_copy[df_copy['company_id']=='cbf1c8b09cd5b549416d49d220a40cbd317f952e']['name'].mode()

0    MiPasajefy
Name: name, dtype: object

In [18]:
# Notamos que al 'name'='MiPasajefy' le corresponde el 'company_id'='cbf1c8b09cd5b549416d49d220a40cbd317f952e'
# por lo tanto, sustituimos en la columna 'name' el valor 'MiPasajefy' cuando 'company_id' es igual a 'cbf1c8b09cd5b549416d49d220a40cbd317f952e'
df_copy.loc[df_copy['company_id']=='cbf1c8b09cd5b549416d49d220a40cbd317f952e', 'name'] = 'MiPasajefy'
# Obtenemos los valores únicos de la columna 'name'
print(df_copy['name'].unique())
print(f"\nValores faltantes en la columna 'name': {df_copy['name'].isnull().sum()}")

['MiPasajefy' 'Muebles chidos']

Valores faltantes en la columna 'name': 0


In [19]:
# obtener la cantidad de filas y columnas
print(f"Shape del Dataframe: {df_copy.shape}\n")
# Mostramos la información del dataframe
df_copy.info()

Shape del Dataframe: (10000, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          10000 non-null  object 
 1   name        10000 non-null  object 
 2   company_id  10000 non-null  object 
 3   amount      10000 non-null  float64
 4   status      10000 non-null  object 
 5   created_at  10000 non-null  object 
 6   paid_at     6009 non-null   object 
dtypes: float64(1), object(6)
memory usage: 547.0+ KB


## Valores nulos

In [20]:
# Vemos si hay valores null en el dataframe
df_copy.isnull().sum()

id               0
name             0
company_id       0
amount           0
status           0
created_at       0
paid_at       3991
dtype: int64

## Valores duplicados en 'id'

In [21]:
# Como 'id' será clave primaria, verificamos que no hay valores duplicados
df_copy['id'].duplicated().sum()

np.int64(0)